In [ ]:
%reload_ext autoreload
%autoreload 2

import os
from glob import glob
import pandas as pd
import QC
from subtle.utils.io import get_dicom_dirs
from tqdm.notebook import tqdm
import logging
import numpy as np
import pydicom
import shutil
logging.getLogger(__name__).setLevel(logging.ERROR)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

QC.util.defaults.QC_MODEL_PATH = '/home/srivathsa/projects/QualityControl/QC/QCapp/models'

def get_binary_qual(qual_results, qual_thresh=0.85):
    score = qual_results.mean(axis=1)
    return score[0][0]

In [ ]:
df_qc = pd.read_csv('/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/qc_scores.csv')
score_dict = df_qc.to_dict(orient='records')
proc_cases = sorted([c['Case'] for c in score_dict])

dcm_base = '/home/srivathsa/projects/studies/gad/stanford/data'
mdi_test = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/fb_192x160'
cases = sorted([f.split('/')[-1] for f in glob('{}/Patient*'.format(mdi_test))])
cases = [c for c in cases if c not in proc_cases]

qual_list = []

for cnum in tqdm(cases, total=len(cases)):
    print('Predicting {}...'.format(cnum))
    try:
        case_dir = os.path.join(dcm_base, cnum)
        t1_pre, t1_post = get_dicom_dirs(case_dir)
        dirs = [d for d in glob('{}/*'.format(case_dir))]
        t2 = [d for d in dirs if 'T2' in d][0]
        flair = [d for d in dirs if 'FLAIR' in d][0]

        print(t1_pre, t1_post, t2, flair)

        qual = QC.predict_image_quality_default([t1_pre, t1_post, t2, flair])
        qual_list.append({
            'Case': cnum,
            'T1': get_binary_qual(qual[0]),
            'T1CE': get_binary_qual(qual[1]),
            'T2': get_binary_qual(qual[2]),
            'FLAIR': get_binary_qual(qual[3])
        })
    except Exception as err:
        print('Cannot process {}:{}'.format(cnum, err))

In [ ]:
df2 = pd.DataFrame(qual_list)
df_qual = pd.read_csv('/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/qc_scores.csv', 
                      index_col=[0])
df_final = pd.concat([df_qual, df2])
df_final.to_csv('/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/qc_scores.csv')

In [ ]:
df_qual = pd.read_csv('/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/qc_scores.csv', 
                      index_col=[0])
df_qual['Mean'] = df_qual[['T1', 'T1CE', 'T2', 'FLAIR']].mean(axis=1)
df_qual.sort_values('Mean')

In [ ]:
df_low_qual = df_qual.query('Mean <= 0.45')
df_low_qual.shape[0]

In [ ]:
all_cases = sorted(df_qual.Case.tolist())
test_cases = sorted(df_low_qual.Case.tolist())
rem_cases = [c for c in all_cases if c not in test_cases]
train_cases = np.random.choice(rem_cases, size=475, replace=False)
val_cases = [c for c in rem_cases if c not in train_cases]

In [ ]:
print(len(train_cases))
print(len(val_cases))
print(len(test_cases))

In [ ]:
base = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/full_brain_256'

for cnum in test_cases:
    shutil.move('{}/{}'.format(base, cnum), '{}/test/{}'.format(base, cnum))
    